In [ ]:
#Extract Covid Cohen Dataset

import numpy as np
import cv2
import pandas as pd
from glob import glob
import os


img_path = 'D:\\University\\Master\\Term 3\\Computer Vision\\HWs\\Project I\\Dataset\\images\*'
csv_path = 'D:\\University\\Master\\Term 3\\Computer Vision\\HWs\\Project I\\Dataset\\metadata.csv'
save_path = 'D:\\University\\Master\\Term 3\\Computer Vision\\HWs\\Project I\\COVID-19 dataset\\Covid'
save_path2 = 'D:\\University\\Master\\Term 3\\Computer Vision\\HWs\\Project I\\COVID-19 dataset\\Normal'

path_img = sorted(glob(img_path))
csv_file = pd.read_csv(csv_path)
flag = 'COVID-19'

for i in range(csv_file.shape[0]):
    
    finding = csv_file['finding'][i]
    if flag in finding:
        filename = os.path.splitext(csv_file['filename'][i])[0]
        for img_path in path_img:
            img_name = os.path.splitext(os.path.basename(img_path))[0]
            if img_name == filename:
                img = cv2.imread(img_path)
                cv2.imwrite(save_path + '\Covid_'+os.path.basename(img_path), img)
    else: 
        filename = os.path.splitext(csv_file['filename'][i])[0]
        for img_path in path_img:
            img_name = os.path.splitext(os.path.basename(img_path))[0]
            if img_name == filename:
                img = cv2.imread(img_path)
                cv2.imwrite(save_path2 + '\\Normal_'+os.path.basename(img_path), img)




In [1]:
import os
import cv2
import numpy as np
from glob import glob
from tqdm import tqdm

def calculate_overall_mean_std(data_path, subset_ratio=0.7, target_size=(224, 224)):
    """
    Calculate the overall mean and standard deviation for 70% of the images from both classes (Covid and Normal).
    
    Args:
    data_path (str): Path to the dataset containing 'Covid' and 'Normal' subfolders.
    subset_ratio (float): Proportion of images to use for calculating statistics (default is 70%).
    target_size (tuple): Target size for resizing images.
    
    Returns:
    - overall_mean (list): Overall mean values for (R, G, B).
    - overall_std (list): Overall standard deviation values for (R, G, B).
    """
    image_paths = []

    # Collect image paths for both classes (Covid and Normal)
    for root, subfolders, filenames in os.walk(data_path):
        for k in subfolders:
            if k == 'Covid' or k == 'Normal':  # Collect paths for both 'Covid' and 'Normal'
                filepath = os.path.join(root, k, "*")
                paths = sorted(glob(filepath))
                image_paths.extend(paths)

    # Shuffle the image paths to ensure randomness
    np.random.shuffle(image_paths)
    
    # Select 70% of the images for training
    train_paths = image_paths[:int(len(image_paths) * subset_ratio)]

    # Initialize variables to calculate mean and std
    pixel_sum = np.zeros(3)
    pixel_sq_sum = np.zeros(3)
    num_pixels = 0

    # Iterate over the selected 70% images
    for img_path in tqdm(train_paths, desc="Calculating Overall Mean and Std"):
        try:
            # Read the image
            img = cv2.imread(img_path)
            if img is None:
                print(f"Could not read image: {img_path}")
                continue

            # Resize the image
            img_resized = cv2.resize(img, target_size)

            # Normalize to [0, 1]
            img_resized = img_resized / 255.0  # Scaling to [0, 1]

            # Accumulate pixel sums and squared sums
            pixel_sum += np.sum(img_resized, axis=(0, 1))
            pixel_sq_sum += np.sum(img_resized**2, axis=(0, 1))
            num_pixels += img_resized.shape[0] * img_resized.shape[1]  # Total pixels per image

        except Exception as e:
            print(f"Error processing {img_path}: {e}")

    # Calculate overall mean and std
    overall_mean = pixel_sum / num_pixels
    overall_std = np.sqrt((pixel_sq_sum / num_pixels) - (overall_mean**2))

    return overall_mean.tolist(), overall_std.tolist()

# Example usage
data_path = r"D:\University\Master\Term 3\Computer Vision\HWs\Project I\Attempt 2\MachineVision Dataset\Dataset"
overall_mean, overall_std = calculate_overall_mean_std(data_path)

print(f"Overall Mean: {overall_mean}")
print(f"Overall Std: {overall_std}")


Calculating Overall Mean and Std: 100%|██████████| 2100/2100 [00:59<00:00, 35.20it/s]

Overall Mean: [0.5113713164952965, 0.5111083020633332, 0.5110829029900134]
Overall Std: [0.2556174130529046, 0.2555650771697803, 0.2555635740709926]
